In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
load_dotenv()
import os 
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
model_name=os.getenv("MODEL")

Model_vision=ChatGoogleGenerativeAI(model=model_name)

c:\Users\AkshayKumarBM\miniconda3\envs\agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from typing import Any, Dict, List, Union, Optional
from pydantic import BaseModel, Field
import re

def json_to_pydantic_model(json_data: Union[List, Dict], class_name: str = "SingleData") -> str:
    """
    Convert a list of field definitions to a Pydantic BaseModel class definition.
    
    Args:
        json_data: List of field definitions [name, type, description] or dict
        class_name: Name for the Pydantic class
        
    Returns:
        String containing the Pydantic class definition
    """
    # Handle list of field definitions
    if isinstance(json_data, list):
        fields = []
        for field_def in json_data:
            if len(field_def) >= 3:
                field_name = field_def[0]
                field_type = field_def[1]
                description = field_def[2]
                fields.append((field_name, field_type, description))
            else:
                return "Error: Each field definition should have name, type, and description."
    else:
        return "Error: Expected a list of field definitions."
    
    # Generate class definition
    class_definition = [
        "from pydantic import BaseModel, Field",
        "from typing import List, Optional, Dict, Any, Union\n",
        f"class {class_name}(BaseModel):"
    ]
    
    for field_name, field_type, description in fields:
        # Sanitize field name to be a valid Python identifier
        field_name = re.sub(r'\W|^(?=\d)', '_', field_name)
        
        # Add field with type annotation and Field description
        class_definition.append(f"    {field_name}: str = Field(default=None, description=\"{description}\")")
    
    return "\n".join(class_definition)



In [6]:
# name
#['str','int','float','bool','List[str]']
# Disction


json_example=[
    ["name","str","name of the person"],
    ["price","int","price for a service "],
    ["gender","bool","return true if he is male"],
    ['list of sujest',"List[str]","list of subjest name"]
]
# Generate the Pydantic class code
pydantic_class_code = json_to_pydantic_model(json_example)
# Execute the generated code to define the class
exec(pydantic_class_code)
print(pydantic_class_code)


from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Union

class SingleData(BaseModel):
    name: str = Field(default=None, description="name of the person")
    price: str = Field(default=None, description="price for a service ")
    gender: str = Field(default=None, description="return true if he is male")
    list_of_sujest: str = Field(default=None, description="list of subjest name")


In [7]:
class Final_data(BaseModel):
    list_data: List[SingleData] = Field(description="List of Objest of given data")

In [8]:
strut_llm=Model_vision.with_structured_output(Final_data)

In [15]:
data="""my data  name is akahsy kumag bm """

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
spitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
spited_data=spitter.split_text(data)
spited_data

['my data  name is akahsy kumag bm']

In [17]:
final_result=[]
for data in spited_data:
    result=strut_llm.invoke(data)
    final_result.append(result)

In [18]:
information=[result.list_data for result in final_result]

In [19]:
from functools import reduce

flattened_list = reduce(lambda x, y: x + y, information)

In [20]:
flattened_list


[SingleData(name='akahsy kumag bm', price=None, gender=None, list_of_sujest=None)]